In [ ]:
# Import required libraries for type hints and LangChain
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
# Test import to verify LangChain is properly installed
from langchain.chat_models import init_chat_model

In [ ]:
# Initialize the language model and set up the chatbot graph

# Create LLM instance using Google's Gemini model
llm = init_chat_model("google_genai:gemini-2.0-flash")

# Define the state structure for the chatbot
class State(TypedDict):
    # Messages list with add_messages reducer to append rather than overwrite
    messages: Annotated[list, add_messages]

# Define the chatbot node that processes messages through the LLM
def chatbot(state: State) -> State:
    """Invoke the LLM with current messages and return updated state"""
    return {"messages": [llm.invoke(state["messages"])]}

# Build the state graph
builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

# Define graph edges
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

# Compile the graph
graph = builder.compile()

In [ ]:
# Test the chatbot with a single question
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# Alternate question to test: "What is the latest price of MSFT stock?"

# Invoke the graph with a single message
response = graph.invoke({"messages":[message]})

# Display the response messages
response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='1c79f3b0-337a-4cf7-8f4e-f3232cb71677'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d6bb6724-3b24-4815-aae1-2ffaed8b0089-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [ ]:
# Interactive multi-turn chatbot conversation
state = None
while True:
    # Get user input
    in_message = input("You: ")
    
    # Exit on quit/exit commands
    if in_message.lower() in {"quit","exit"}:
        break
    
    # Initialize or update state with user message
    if state is None:
        # First message: create new state
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        # Subsequent messages: append to existing state
        state["messages"].append({"role": "user", "content": in_message})

    # Invoke graph and get response
    state = graph.invoke(state)
    
    # Print the bot's response (last message in the conversation)
    print("Bot:", state["messages"][-1].content)